In [72]:
import os
import regex
import shutil
import random
from tqdm import tqdm
from shutil import copyfile

In [2]:
TXT_DIR = "theatregratuit-txt"
if os.path.isdir(TXT_DIR):
    fnames = [x for x in os.listdir(TXT_DIR) if ".txt" == os.path.splitext(x)[-1]] # don't load vim *.swp files
    n_files = len(fnames)

In [102]:
TMP_DIR = "tmp1"
if not os.path.isdir(TMP_DIR):
    os.mkdir(TMP_DIR)

In [101]:
TMP_DIR2 = "tmp2"
if not os.path.isdir(TMP_DIR):
    os.mkdir(TMP_DIR)

In [4]:
if os.path.isdir(TMP_DIR):
    ffnames = [x for x in os.listdir(TXT_DIR) if ".txt" == os.path.splitext(x)[-1]] # don't load vim *.swp files
    n_ffiles = len(fnames)    

In [132]:
def check_create_dir(d, clean=False):
    if not os.path.isdir(d):
        os.mkdir(d)
    if clean:
        [os.remove(os.path.join(d,f)) for f in os.listdir(d)]

In [134]:
def get_filenames(d):
    if os.path.isdir(d):
        fnames = [x for x in os.listdir(d) if ".txt" == os.path.splitext(x)[-1]] # don't load vim *.swp files
        return fnames
    else:
        print("dir not found:", d)

---

In [7]:
def printlines(lines):
    print('\n'.join(lines))

In [8]:
def printsep(s):
    print(s*40)

In [9]:
def underprint(x):
    print(x)
    print('-'*len(x))

In [10]:
def print_file(f):
    underprint(f)
    print()
    raw, lines, lines_len = get_lines(f)
    printlines(lines)

In [11]:
def print_random_file(files):
    total = len(files)
    ind = random.randint(0, total-1)
    f = files[ind]
    print(f"file {ind}/{total}")
    print_file(f)

---

In [12]:
def get_lines(input_name, le_dir=TMP_DIR):
    with open(os.path.join(le_dir, input_name), "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read()
        lines = [l.strip() for l in raw.split('\n')]
    return raw, lines, len(lines)

In [13]:
def get_data(f, verbose=True, le_dir=TMP_DIR):
    if verbose:
        underprint(f)
        print()
    raw, lines, lines_len = get_lines(f, le_dir=le_dir)
    data = {
        "fname": f,
        "raw": raw,
        "lines": lines,
        "lines_len": lines_len,

    }
    return data

In [14]:
def get_all_files_data(le_dir=TMP_DIR):
    fnames = get_filenames(le_dir)
    all_data = {}
    for f in fnames:
        all_data[f] = get_data(f, verbose=False)
    return all_data

In [15]:
ALL_DATA = get_all_files_data()

---

## The meaty part: reformatting

In [17]:
def split_by_regex(fnames, pattern, threshold=1, verbose=True):
    with_pattern = []
    without_pattern = []
    for fname in fnames:
        found = 0
        raw, lines, lines_len = get_lines(fname)
        for i, l in enumerate(lines):
            if regex.search(pattern, l):
                found += 1
                if found == threshold:
                    break
        if found == threshold:
            with_pattern.append(fname)
        else:
            without_pattern.append(fname)
    if verbose:
        print("with:", len(with_pattern))
        print("without:", len(without_pattern))
    return with_pattern, without_pattern

In [28]:
def at_least_n_matches(lines, pattern, threshold=1):
    found = 0
    for l in lines:
        if regex.search(pattern, l):
            found +=1
            if found == n:
                break
    if found == threshold:
        return True
    else:
        return False

In [36]:
def get_matches(lines, pattern, threshold=1):
    found = 0
    matches = []
    for i, l in enumerate(lines):
        m = regex.search(pattern, l)
        if m:
            found +=1
            matches.append((i, m))
    if found >= threshold:
        return matches
    else:
        return False

---

Replace only when the regex is found enough times (how much is determined empirically).

The thing we want to find:
- char followed by space and colon;
- char followed by ". — "
- didascaliae:
 - sometimes in parentheses
 - most of the time with commas
 

---

In [187]:
def make_regices():
    return {
        "blank_line": regex.compile("^\s*$"),
        "blank_line_with_rubbish": regex.compile("^[\p{Z}\p{P}]*$"),
        
        # trim beginning & end of files
        # -----------------------------
        "character": regex.compile(
            "^((les\s*)*pe*rsonna *ge|(les)*acteurs|dramatis personae|avertissement|entreparleur|biographies|apparences|personrage|persongueules|pépersonâge)",
            regex.IGNORECASE,
            ),
        "author": regex.compile("^de$",regex.IGNORECASE),
        "additional_author": regex.compile("^(\(Auteur inconnu\)|" +\
                                           "Voltaire|" +\
                                           "de\s+Georges Courteline|" +\
                                           "Translation en prose de Jean Sibil|" +\
                                           "Anton Pavlovitch Tchekhov|" +\
                                           "EUGENE LABICHE, A. LEFRANC et MARC-MICHEL|" +\
                                           "Alexandre Hardy)$"),
        "la_scene": regex.compile("^La scène"),        
        "fin": regex.compile("(fin|rideau|f1n|inachev|manque)", regex.IGNORECASE),
        
        
        # search for lines made of full caps + punct/space, as well as some
        # very common words listing characters, and didascalia in ()
        "LINE": regex.compile("^((\p{Lu}+|seule?|puis|puis tout le monde|moins|et)[\p{Z}\p{P}]*)+\p{Z}*" +\
                              "(\(.*?\))*\p{Z}*$"),
        "(line)": regex.compile("^\(.*\)$"),
        
        # common annoying starts "M.", "L'"
        "annoying_init": regex.compile("^[\p{P}\p{Z}]*(M\.|\p{Lu}')\p{Z}*"),
        # words (with possible - or '), and space, with punctuation at the end
        "caps_init": regex.compile("^\s*(([\p{Lu}\p{Pc}\p{Pd}1]+\p{Zs}*)+)(\s*[\p{Po}]\s*)"),
        "char_lc_dot_dash": regex.compile("^(\p{Z}*M*[\p{Ll}\p{Z}]+\.)\p{Z}*\p{Pd}+\p{Z}*(.*)$"),
        "trailing_space": regex.compile("\s*$"),
        "trailing_punct": regex.compile("\s*[.,:]*\s+$"),
        "non_breaking_space": regex.compile(" {2,}"), # non-breaking space
        ".": regex.compile("\p{Z}*\.\p{Z}"),
        ",": regex.compile("\p{Z}*,\p{Z}*"),
        "final_dot": regex.compile("\s*\.\s*$"),
        "final_comma": regex.compile(",$"),
        
        # include rare errors like ".-—Blah" or ". -— Blah"
        "dot_n_dash": regex.compile("[,.]\p{Z}*\p{Pd}+\p{Z}*"),        
        "dot_opt_dash": regex.compile("\p{Z}*\.\p{Z}*\p{Pd}*\p{Z}*"),
        "comma_dash": regex.compile("\p{Z}*,\p{Z}*\p{Pd}\p{Z}*"),

        
        "dash_and_more": regex.compile("(.*?)[.;:,]*\s*-*[-–—]\s*"),
        "colon": regex.compile("\s*:\s*"),
        "colon_and_more": regex.compile("(.*?)\s*:\s*"),
        "didasc_and_more": regex.compile(".*?(?<!M)[.:]\s*"),
        
        # "char_no_dot": regex.compile("^\s*([A-Z1'-]+)\s*$"),
        # # using all 3 lengths: -, –, —, bc of inconsistencies
        # "char_dash": regex.compile("^\s*([A-Z1'-]{2,})\.[\s-]+[–—]\s"),
        # "char_comma_dash": regex.compile("^\s*([A-Z1'-]{2,},.*?)\.[\s-]+[–—]\s"),
        # "char_dot_more": regex.compile("^\s*([A-Z1'-]{2,}\.)(.*)$"),
        # "char_no_dot_more": regex.compile("^\s*([A-Z1'-]{2,})(.*)$"),
        
        
        "WORD": regex.compile("\p{Lu}{2,}"), 
        "WORD_INIT": regex.compile("^(?:\p{Lu}[,']\p{Z}*)*[\p{Lu}\p{Pd}]{2,}"),         
        "CHAR_ONELINE": regex.compile("^\p{Lu}+[\p{Z}\p{P}]*$"),
        "CHAR_ONELINE_DOT": regex.compile("^\p{Lu}+\p{Z}*\.\p{Z}*$"),
        "CHAR_ONELINE_COLON": regex.compile("^\p{Lu}+\p{Z}*:\p{Z}*$"),
        "INIT_char_DOT_n_DASH" : regex.compile("^\p{L}+\.\p{Z}+\p{Pd}"),
        "INIT_char_COLON" : regex.compile("^\p{L}+\p{Z}+:"),          
        "INIT_CHAR_DOT_N_DASH" : regex.compile("^\p{Lu}+\.\p{Z}+\p{Pd}"),
        "INIT_CHAR_COLON" : regex.compile("^\p{Lu}+\p{Z}+:"),      
        "init_char_dot_n_dash" : regex.compile("^\p{Lu}+\p{Ll}+\.\p{Z}+\p{Pd}"),
        "init_char_colon" : regex.compile("^(\p{Lu}\p{Ll}+)\p{Z}*:\p{Z}*"),

        # single file with "A. or B. as characters: count-3202657-De_la_liberte.txt"
        "LETTER_CHARS": regex.compile("^(\p{Lu}\.)\p{Z}\p{Pd}\p{Z}"),       
        
        # regices for innards cleaning
        # ----------------------------
        # letters & space, possibly a parenthesis, : or — at the end
        "char_colon_or_dash": regex.compile("^((?:\p{Lu}[,']\p{Z}*)*" + \
                                            "[\p{L}\p{Pd}\p{Z}]+)" +\
                                            "(\p{Z}\(.*?\))*" +\
                                            "\p{Z}[:—]\p{Z}"),
        
        # lower case character, optionally with didasc, ending in dot and dash, 
        "char_lc_dot_dash": regex.compile("^([\p{Ll}\p{Z}]+)" +\
                                          "(([,\p{Z}].*?)*)" + \
                                          "\.\p{Z}*\p{Pd}\p{Z}*"),
        
        # char (with numbers & spaces) followed or not by :
        "char_oneline_colon": regex.compile("^((L')*[\p{L}\p{N}\p{Z}]+?)\p{Z}*:$"),
        
        "char_colon": regex.compile("^(\p{L}{2,})\p{Z}*:\p{Z}*"),
        # "char_colon_more": regex.compile("^\s*([A-Z1'-]{2,})\s*:\s*(.*)$"),

        # line cleanup
        "(note)": regex.compile("\(\d+\)"),
        "sc_ene": regex.compile("SC\p{Z}+[EÈ]NE", regex.IGNORECASE),
        "deuxi_eme": regex.compile("DEUXI\p{Z}ÈME", regex.IGNORECASE)
    }

R = make_regices()

In [191]:
limit = 1
total = len(ffnames)

check_create_dir(TMP_DIR2, clean=True)

for i, (f, data) in enumerate(ALL_DATA.items()):

    if i == limit:
        break
        
    underprint(f)

    formatted = []
    
    # skip first match, so that entire start of text caught in one extract 
    # (adding <|s|> at the very top of file after the loop)    
    skipped = False
    
    for i, l in enumerate(data["lines"]):
        
        # remove footnote calls (digit)
        l = regex.sub(R["(note)"], "", l)

#         colon_re = regex.search(R["char_colon_or_dash"], l)
#         if colon_re:
#             formatted, skipped = skip_first_case(formatted, skipped)
#             start = colon_re.group(1).upper()
#             if colon_re.group(2):
#                 start += colon_re.group(2)
#             start += "."
#             end = l[colon_re.span()[1]:]       
#             formatted.append(start)
#             if end: 
#                 formatted.append(end)
#             continue

#         char_lc_dot_dash = regex.search(R["char_lc_dot_dash"], l)
#         if char_lc_dot_dash:
#             formatted, skipped = skip_first_case(formatted, skipped)
#             start = char_lc_dot_dash.group(1).upper()
#             if char_lc_dot_dash.group(2):
#                 start += char_lc_dot_dash.group(2)    
#             start += "."
#             end = l[char_lc_dot_dash.span()[1]:]       
#             formatted.append(start)
#             if end: 
#                 formatted.append(end)
#             continue    

#         char_oneline_colon = regex.search(R["char_oneline_colon"], l)
#         if char_oneline_colon:
#             formatted, skipped = skip_first_case(formatted, skipped)
#             start = char_oneline_colon.group(1).upper()    
#             start += "." 
#             end = l[char_oneline_colon.span()[1]:]               
#             formatted.append(start)
#             if end: 
#                 formatted.append(end)
#             continue         
        
        # check for full caps line
        caps_line = regex.match(R["LINE"], l)
        if caps_line and not regex.match(R["WORD_INIT"], formatted[-1]):
            formatted, skipped = skip_first_case(formatted, skipped)
            formatted.append(l)
            continue
            
        # init capital letters word
        word_init = regex.match(R["WORD_INIT"], l)
        # check that previous line is not already with full caps        
        if word_init and not regex.search(R["WORD_INIT"], formatted[-1]):
            # is there a dot'n'dash straight away?
            rest = l[word_init.span()[1]:]            
            dot_n_dash = regex.match(R["dot_n_dash"], rest)
            if dot_n_dash:
                start = l[:word_init.span()[1]] + "."          
                rest = l[word_init.span()[1]+dot_n_dash.span()[1]:]
                formatted, skipped = skip_first_case(formatted, skipped)
                formatted.append(start)                
                formatted.append(rest)                
                continue
            else:
                # get all caps words on the line, and check the last one
                more_caps_words = regex.finditer(R["WORD"], l)
                # Find the last one.
                # https://stackoverflow.com/a/2988680
                for last_caps_word in more_caps_words: pass
                if last_caps_word:
                    formatted, skipped = skip_first_case(formatted, skipped)
                    start = l[:last_caps_word.span()[1]] + "."
                    rest = l[last_caps_word.span()[1]:]
                    formatted.append(start)                
                    if rest:
                        # check for comma and dash
                        comma_n_dash = regex.match(R["comma_dash"], rest)
                        if comma_n_dash:
                            formatted.append(rest[comma_n_dash.span()[1]])
                        # check for didascalia: find either dot or dot'n'dash later               
                        dot_or_dash = regex.search(R["dot_opt_dash"], rest)
                        if dot_or_dash:
                            formatted.append(rest[dot_or_dash.span()[1]:])
                        # if dot'n'dash, find the end of that
                        more_caps_words, last_caps_word = None, None # needs reset        
                    continue
            
        # otherwise just append the line
        formatted.append(l)
        
    # add very first and very end markers, trim lines        
    formatted = markers_final_cleanup(formatted)

    # save lines
    data["formatted"] = formatted

    save_lines(data["formatted"])

#     printlines(lines)
#     printsep('-')
#     printlines(data["formatted"])


count-2454059-Champignol_malgre_lui.txt
---------------------------------------


In [145]:
def skip_first_case(lines, skipped):
            if skipped:
                lines = append_markers(lines) 
            return lines, True 

---

## Layout

In [63]:
def append_markers(lines):
    lines.append("<|e|>")
    lines.append("<|s|>")
    return lines

In [65]:
def insert_markers(lines, ind):
    lines.insert(ind, "<|e|>")
    lines.insert(ind+1, "<|s|>")

In [97]:
def insert_formatted(lines, line, match, ind):
    lines.insert(ind, "<|e|>")
    lines.insert(ind+1, "<|s|>")
    lines.insert(ind+2, match.group(1).upper() + ".")
    # if text after the match, add it to the next line
    if not regex.search(R["blank_line_with_rubbish"], line[match.span()[1]:]):
        lines.insert(ind+3, line[match.span()[1]:])

In [40]:
def remove_trailing_lines(lines):
    if regex.match(R["blank_line"], lines[-1]):
        for i, l in enumerate(reversed(lines)):
            if not regex.match(R["blank_line"], l):
                return lines[:-i]
    else:
        return lines

In [41]:
def markers_final_cleanup(lines):
    lines = ["<|s|>"] + lines   
    lines = remove_trailing_lines(lines)
    lines.append("<|e|>") 
    return lines

---

In [103]:
def save_lines(lines, le_dir=TMP_DIR2):
    check_create_dir(le_dir)
    with open(os.path.join(le_dir, f), 'w') as o:
        o.write('\n'.join(lines))

In [181]:
def save_split_groups(reg, threshold=1, verbose=False, le_dir=TMP_DIR, le_dir1=TMP_DIR2, le_dir2=TMP_DIR3):
    get_filenames(le_dir)
    check_create_dir(le_dir1, clean=True)
    check_create_dir(le_dir2, clean=True)
    with_it, without = split_by_regex(ffnames, reg, threshold=threshold, verbose=verbose)    
    for f in tqdm(with_it, desc=f"files with in {le_dir1}   "):
        copyfile(os.path.join(le_dir, f), os.path.join(le_dir1, f))
    for f in tqdm(without, desc=f"files without in {le_dir2}"):
        copyfile(os.path.join(le_dir, f), os.path.join(le_dir2, f))        

---

In [54]:
short_files = []
for f in ffnames:
    d = get_data(f, verbose=False)
    if d["lines_len"] < 20:
        short_files.append(f)
print(len(short_files))

24


In [65]:
print_random_file(short_files)

file 10/24
count-2773488-Dialogues_des_morts_7_Romulus_et_Remus.txt
--------------------------------------------------------

7. ROMULUS et REMUS

La grandeur à laquelle on ne parvient que par le crime ne saurait donner ni gloire ni bonheur solide.

REMUS. Enfin, vous voilà, mon frère, au même état que moi; cela ne valait pas la peine de me faire mourir. Quelques années où vous avez régné seul sont finies; il n'en reste plus rien; et vous les auriez passées plus doucement si vous aviez vécu en paix, partageant l'autorité avec moi.
ROMULUS. Si j'avais eu cette modération, je n'aurais ni fondé la puissante ville que j'ai établie, ni fait les conquêtes qui m'ont immortalisé.
REMUS. Il valait mieux être moins puissant, et être plus juste et plus vertueux; je m'en rapporte à Minos et à ses deux collègues qui vont vous juger.
ROMULUS. Cela est bien dur. Sur la terre personne n'eût osé me juger.
REMUS. Mon sang, dans lequel vous avez trempé vos mains, fera votre condamnation ici-bas, et sur l

---

### Splitting dataset into similar files

In [180]:
save_split_groups(R["init_char_dot_n_dash"], threshold=2)

files without in tmp3: 100%|██████████| 1053/1053 [00:00<00:00, 5785.88it/s]


In [110]:
caps, no_caps = split_by_regex(ffnames, R["WORD"])

with: 1038
without: 45


In [31]:
caps, no_caps = split_by_regex(ffnames, R["WORD_INIT"])

with: 1012
without: 71


In [52]:
print_random_file(no_caps)

file 56/71
count-1541294-Olaf_loriginal.txt
--------------------------------

(Catherine I)
Un salon-salle à manger au mobilier à la fois en harmonie et hétéroclite. Superbes rideaux. Il règne un certain laisser-aller : revues diverses en vrac dans un coin, coussins mal placés sur le divan, fauteuils mal placés (deux sont vis-à-vis pour étendre ses jambes sur l'autre)... Aux murs, très soigneusement accrochés, des photos de mode, des affiches de concerts exceptionnels (toutes musiques), des posters de films récents... Effet de surcharge.
Olaf (la cinquantaine sportive, tenue de footing, une petite queue de cheval et une mèche bleue sur le devant que, debout vers la cheminée, il examine dans une glace de poche) : Je sentais que l'orange me siérait mieux.
Corinne (la trentaine, avachie dans ses fauteuils vis-à-vis, en robe de chambre à grosses fleurs) : Chéri, je t'aime de toutes les couleurs.
Olaf : Tu t'en tires toujours comme ça au lieu de donner des conseils utiles.
Corinne (agressiv

In [66]:
caps_w, no_caps_w = split_by_regex(ffnames, R["WORD"], threshold=1)

with: 1038
without: 45


In [104]:
t, no_t = split_by_regex(no_caps_w, R["char_colon_or_dash"], threshold=5)

with: 15
without: 30


In [106]:
print_random_file(no_t)

file 5/30
count-1788231-LASSEMBLEE_DES_FEMMES.txt
---------------------------------------

Praxagora :
Eh, oeil luisant de lampe d'art, au dessin si réussi, nous allons expliquer ta raison d'être et ton rôle, car sous l'impulsion créative du potier tu as des trous de lumière vive comme le soleil. Envoie les signaux lumineux convenus. A toi seule nous livrons l'explication; c'est justice parce que dans nos chambres tu te tiens à nos côtés quand nous nous exerçons aux jeux d'Aphrodite, et personne ne chasse des maisons ton œil regardant nos corps se cambrer. Seule tu éclaires jusqu'aux recoins intimes de nos cuisses dont tu incendies la toison. Et puis tu nous assistes quand nous entrons en cachette dans les celliers pleins des récoltes et de vin. En bonne complice tu restes muette avec les voisins. Aussi vas-tu tout connaître de nos projets, tout ce que mes amies et moi avons décidé à la fête des Parasols. Mais personne, malgré leurs engagements. Pourtant l'aube approche, la Chambre va 

In [586]:
test, no_test = split_by_regex(ffnames, R["char_oneline_colon"], threshold=20)

with: 56
without: 1027


In [597]:
print_random_file(test)

file 39/56
count-2683966-MARINO_FALIERO.txt
--------------------------------

La scène est à Venise, en 1355.

ACTE PREMIER.
SCÈNE I.
(L'appartement du doge.)
ELENA. (Elle est assise et brode une écharpe.)
Une écharpe de deuil, sans chiffre, sans devise!
Hélas ! triste présent ! mais je l'avais promise,
Je devais l'achever... Vaincu par ses remords,
Du moins après ma faute, il a quitté nos bords;
Il recevra ce prix de l'exil qu'il s'impose.
(Elle se lève et s'approche de la fenêtre.)
Le beau jour ! que la mer où mon œil se repose,
Que le ciel radieux brillent d'un éclat pur,
Et que Venise est belle entre leur double azur!
Lui seul ne verra plus nos lagunes chéries :
Il n'est qu'une Venise ! on n'a pas deux patries !...
Je pleure... Oui, Fernando, sur mon crime et le tien.
Pourquoi pleurer? j'ai tort : les pleurs n'effacent rien.
Mon bon, mon noble époux aime à me voir sourire;
Eh bien, soyons heureuse, il le faut...
Je veux lire.
Le Dante, mon poète ! essayons... je ne puis.
Nous le li

In [ ]:
test, no_test = split_by_regex(ffnames, R["INIT_char_COLON"])

In [641]:
char_oneline, no_char_oneline = split_by_regex(ffnames, R["CHAR_ONELINE"], threshold=10)

with: 545
without: 538


In [642]:
print_random_file(no_char_oneline)

file 302/538
count-2877354-BELLAVITA.txt
---------------------------

De nos jours.

La scène représente un salon dans la maison de l'avocat CONTENTO. L'entrée est au fond et donne sur un corridor. Une porte à droite pour l'appartement de l'avocat. Deux portes à gauche. La première communique avec la salle d'attente, l'autre avec le cabinet de l'avocat.

Au lever du rideau, LE SECRETAIRE, jeune, vêtu modestement, mais avec des prétentions à l'élégance, tête un peu vitrine de coiffeur posée sur un cou trop long, s'efface pour laisser passer le notaire DENORA, gras, chauve, la quarantaine, le poil roux, large visage couperosé.
LE SECRETAIRE. — Asseyez-vous, monsieur le notaire.
DENORA, sombre, contenant à grand-peine le tourment qui le dévore. — Il faudra attendre longtemps?
LE SECRETAIRE. — Un petit moment, je le crains. Mais je cours prévenir Madame.
(Il se dirige vers la porte à droite.)
DENORA, le retenant. — Mais non, laissez donc Madame tranquille. Pourquoi déranger Madame?
LE SECR

In [126]:
char_oneline_dot, char_oneline_nodot = split_by_regex(char_oneline, R["CHAR_ONELINE_DOT"])

with: 432
without: 249


In [618]:
char_oneline_colon, char_oneline_nocolon = split_by_regex(fnames, R["CHAR_ONELINE_COLON"], threshold=10)

with: 1
without: 1082


In [619]:
print_random_file(char_oneline_colon)

file 0/1
count-1300100-Occupe.txt
------------------------

SCENE PREMIERE
AMELIE,  BIBICHON,  PALMYRE, YVONNE, VALCREUSE, BOAS, puis ETIENNE
Au lever du rideau, Amélie est debout, près du piano, en train de faire entendre le gramophone à ses invités. Bibichon, un cigare à la bouche, est assis sur le canapé entre Palmyre (1) et Yvonne (3). (Palmyre est assise sur le bras du canapé.) Valcreuse, dos au public, et Boas, face au public, sont assis à la table à jeu, en train de faire une partie de cartes. Le gramophone est en marche, exécutant un grand air chanté par Caruso. On écoute religieusement avec des dodelinements de tête extasiés. (Le morceau chanté par Caruso est l'air d'Il Trovatore : Di quella pira... enregistré par la Société des gramophones. Mettre le disque en mouvement, le rideau encore baissé, et ne lever qu'à la fin de la huitième mesure de chant après la ritournelle, à Marse avvanpo.)
YVONNE, sur un port de voix à effet de Caruso, à la treizième ou quatorzième mesure. Oh!

In [112]:
init_char_dot_n_dash, no_init_char_dot_n_dash = split_by_regex(fnames, R["INIT_CHAR_DOT_N_DASH"])

with: 392
without: 691


In [82]:
fnames_it = iter(with_it)
# fnames_it = iter(without)

In [111]:
fname = next(fnames_it)
print_file(fname)

count-2877388-ROCAMBOLLE_LE_BATELEUR.txt
----------------------------------------

La scène se passe à Bayonne, en 1803.

ACTE I
Le théâtre représente une place publique. A gauche, un cabaret avec deux tables en plein vent. A droite, le derrière de la baraque de ROCAMBOLLE, espèce de tente fermée avec des rideaux mobiles; on lit au-dessus de la porte :
ENTRÉE PABTICULIÈRE DES ACTEURS. LE PUBLIC N'ENTRE PAS ICI.
SCÈNE PREMIÈRE.
MIC-MAC, CHALAMEL à la table placée au premier plan, SOLDATS, à la deuxième table, deuxième plan.
CHŒUR.
Enfants d'la giberne,
Noyons dans l'cognac
Les chagrins d'caserne,
L'ennui du bivouac.
Honte à c'lui qui bouge
Et qui r'cule d'un ch'veu
D'vant un habit rouge,
Ou d'vant un vin bleu.
Enfants, etc., etc.
MIC-MAC, qui a de grands cheveux blonds, trinquant avec CHALAMEL. — A vot’santé, militaire et la société.
CHALAMEL. — A la vôtre et à celle du premier Consul. (Tous se lèvent et trinquent. — Se rasseyant.) Qu'est-ce que vous cachez donc dans ce sac-là, monsieur

In [114]:
init_char_colon2, no_init_char_colon2 = split_by_regex(init_char_dot_n_dash, R["INIT_CHAR_COLON"])

with: 112
without: 280


In [113]:
init_char_colon, no_init_char_colon = split_by_regex(no_init_char_dot_n_dash, R["INIT_CHAR_COLON"])

with: 88
without: 603


In [118]:
with_it, without = split_by_regex(fnames, R["init_char_dot_n_dash"])

with: 14
without: 1069


In [117]:
with_it, without = split_by_regex(fnames, R["init_char_colon"])

with: 8
without: 1075


---

---
Some files have nothing in caps after trimming.

In [1868]:
with_it, without = split_by_regex(fnames, r"^\w+\s*:")

with: 454
without: 629


In [1879]:
fnames_it = iter(with_it)
# fnames_it = iter(without)

In [1880]:
fname = next(fnames_it)
print_file(fname)

count-2454059-Champignol_malgre_lui.txt
---------------------------------------

Le 1er acte à Paris dans l'hôtel de CHAMPIGNOL.
Le 2* acte à Clermont.
Le 3* acte chez Mme Rivolet, dans les environs de Clermont.

ACTE I
A droite la chambre de madame CHAMPIGNOL. — A gauche, premier plan, porte donnant sur les appartements. — Au deuxième plan, grande fenêtre, porte au fond, donnant sur l'antichambre. — Au fond de l'antichambre, porte donnant sur l'escalier. — Tableaux sur des chevalets. — Etudes sur les murs, etc. - A droite, premier plan, une table flanquée de deux chaises, sur la table une tasse de chocolat servie; à gauche, un canapé, et, à côté du canapé et à droite, deux chaises volantes. — Au fond contre le mur, une toile placée sur une chaise et retournée.
SCENE PREMIERE
SAINT-FLORIMOND, puis ANGELE.
Au lever du rideau, la scène est vide. Un coucou, sonne huit heures, puis on entend une clef tourner dans la serrure de la porte d'entrée, au fond de l'antichambre, qui s'ouvre et SAI

Le dash following chars. 

In [1811]:
with_dash, without_dash = split_by_regex(fnames, R["dot_n_dash"])

with: 777
without: 306


In [1812]:
# fnames_it = iter(caps_word)
fnames_it = iter(no_caps_word)

In [1852]:
fname = next(fnames_it)
print_file(fname)

count-2488208-Une_passade_de_Donata.txt
---------------------------------------

Un grand salon avec un escalier, les volets sont baissés (fermés). Un couple en tenue de ville, Xavier et Irène Gevrais, elle la trentaine, lui la quarantaine; un homme en tenue plus décontractée, la trentaine, Robert.

1. Xavier : En haut ?
Robert : Elle va descendre.
Irène : Elle va d'abord nous faire attendre. Jusqu'au moment où on sera exaspérés. Je connais ses raffinements.
Robert : Elle a changé.
Xavier (railleur) : C'est une grande actrice; Donata change souvent.
Robert : Elle a renoncé à son art.
Irène : Foutaise.
Xavier : Vous nous avez bien annoncés comme je vous avais dit ?
Robert : Oui oui. Je ne connais pas votre nom de toute façon. "L'homme à l'impeccable complet."
Irène : Et mon chapeau ? Vous lui avez parlé de mon nouveau chapeau ? (Il est très élégant et sobre; rien de comique.)
Robert : A une époque où l'on n'en porte plus guère, je n'aurais eu garde d'oublier. Curieusement cela a semblé 